Fake News Classifier Using LSTM  
Dataset:- https://www.kaggle.com/c/fake-news/data

In [1]:
import pandas as pd


In [2]:
#It is done encoding problem
df=pd.read_csv("train.csv",error_bad_lines=False,engine="python")

<ipython-input-2-999b3a976366>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv("train.csv",error_bad_lines=False,engine="python")
Skipping line 810: unexpected end of data


In [3]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
803,803,North Korean Is Arrested in Killing of Kim Jon...,Richard C. Paddock and Rick Gladstone,"KUALA LUMPUR, Malaysia — A man from North K...",0
804,804,SURGEONS ADMIT THAT MAMMOGRAPHY IS OUTDATED AN...,newsguy76,Home › HEALTH › SURGEONS ADMIT THAT MAMMOGRAPH...,1
805,805,Le terrorisme régional protège Israël,NaN,Le terrorisme régional vise à protéger les int...,1
806,806,WAR ON THE STREETS OF PARIS: Armed migrants fi...,wmw_admin,"By wmw_admin on November 3, 2016 Nick Gutterid...",1


In [4]:
df.head()#If 1 then it is fake news classifier and if 0 not a fake news

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.shape

(808, 5)

In [ ]:
# we are going to take our title and train it (lstm)

In [6]:
#check whether we have any null data set or not and add them all
df.isnull().sum() # null value is present

id         0
title     21
author    85
text       2
label      0
dtype: int64

In [7]:
# now in this case what should we do
# 1 - drop the values or
# 2 - replace the values in a better way
# Since it is text data , so we cannot replace the value
# so we will drop the null values
df=df.dropna()

In [8]:
df.shape #see the shape change

(702, 5)

In [9]:
# we going to work with the title because of lstm
# all the independent features
x=df.drop('label',axis=1) # drop label column of the x axis

In [10]:
x

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
802,802,Trump Team’s Links to Russia Crisscross in Was...,Scott Shane and Andrew E. Kramer,"WASHINGTON — During the 2016 campaign, Dona..."
803,803,North Korean Is Arrested in Killing of Kim Jon...,Richard C. Paddock and Rick Gladstone,"KUALA LUMPUR, Malaysia — A man from North K..."
804,804,SURGEONS ADMIT THAT MAMMOGRAPHY IS OUTDATED AN...,newsguy76,Home › HEALTH › SURGEONS ADMIT THAT MAMMOGRAPH...
806,806,WAR ON THE STREETS OF PARIS: Armed migrants fi...,wmw_admin,"By wmw_admin on November 3, 2016 Nick Gutterid..."


In [11]:
## get dependent features
y=df['label']

In [12]:
y

0      1
1      0
2      1
3      1
4      1
      ..
802    0
803    0
804    1
806    1
807    0
Name: label, Length: 702, dtype: int64

In [13]:
x.shape

(702, 4)

In [14]:
y.shape

(702,)

In [15]:
import tensorflow as tf

In [16]:
tf.__version__

'2.13.0'

In [17]:
from tensorflow.keras.layers import Embedding # embeddinglayer - convert word into vectors
from tensorflow.keras.preprocessing.sequence import pad_sequences # it create a padding layer
from tensorflow.keras.models import Sequential # sequential - A Sequential model is where we can perform forward and backward propagation
from tensorflow.keras.preprocessing.text import one_hot # one hot encodong
from tensorflow.keras.layers import LSTM #lstm
from tensorflow.keras.layers import Dense # dense -Just your regular densely-connected NN layer.

In [18]:
### vocabulary size
voc_size=5000 #ohe will be big based on this
# now use the title column and use it

OHE Representation

In [19]:
messages=x.copy() # I will not work on x because it contain all the independent features

In [20]:
messages['title'][2] # 1 - It is the index position

'Why the Truth Might Get You Fired'

In [21]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
802,802,Trump Team’s Links to Russia Crisscross in Was...,Scott Shane and Andrew E. Kramer,"WASHINGTON — During the 2016 campaign, Dona..."
803,803,North Korean Is Arrested in Killing of Kim Jon...,Richard C. Paddock and Rick Gladstone,"KUALA LUMPUR, Malaysia — A man from North K..."
804,804,SURGEONS ADMIT THAT MAMMOGRAPHY IS OUTDATED AN...,newsguy76,Home › HEALTH › SURGEONS ADMIT THAT MAMMOGRAPH...
806,806,WAR ON THE STREETS OF PARIS: Armed migrants fi...,wmw_admin,"By wmw_admin on November 3, 2016 Nick Gutterid..."


In [22]:
# reset the index if incase there is a mismatched
messages.reset_index(inplace=True)

In [ ]:
# we have to apply ohe on the title

In [23]:
import nltk # put ?? to get the documentation
import re #re-reguler expression
from nltk.corpus import stopwords

In [24]:
# we are going to download all the stop words
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# we are first going to do stopwords and lemmatization on the title

In [25]:
### Data Preprocessing
from nltk.stem.porter import PorterStemmer #stemming purpose
ps=PorterStemmer()
corpus=[] #below this all will be in the corpus
for i in range (0,len(messages)):
  review=re.sub('[^a-zA-Z]',' ' , messages['title'][i]) # remove all the special character and replace it with space
  review=review.lower()
  review=review.split() # words by words

  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' ' . join(review) # join all the review
  corpus.append(review) #combine all in the corpus
# review will have list of words
# and if the word is not in stopwords (english) we have to stem it
# if it is present in stopwords I have to ignore
#re.sub- returns all the string value

In [26]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [27]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
#we can apply lemmatization but it will take more time

In [28]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [29]:
# ohe
onehot_rep=[one_hot(words,voc_size)for words in corpus] #index will be captured
onehot_rep

[[966, 3752, 3249, 2737, 1529, 1660, 3403, 4781, 1896, 3750],
 [4217, 170, 1678, 379, 4228, 779, 1604],
 [364, 4962, 3425, 3356],
 [4408, 1165, 3040, 2948, 2671, 2839],
 [3993, 4228, 1164, 1646, 1717, 2006, 4228, 2352, 1336, 1615],
 [3028,
  1926,
  3637,
  3541,
  2270,
  2392,
  3167,
  1191,
  150,
  1566,
  620,
  1220,
  1156,
  4590,
  1604],
 [1018, 2768, 2339, 871, 812, 4268, 560, 4144, 413, 4494, 556],
 [233, 1847, 21, 3852, 8, 2079, 2392, 368, 413, 4494, 556],
 [2074, 3524, 3892, 669, 3543, 4031, 4697, 4008, 2392, 4091],
 [2379, 2991, 3135, 3269, 1493, 2961, 1707, 4627],
 [1976, 3932, 1642, 3022, 1861, 1591, 1984, 825, 3993, 2399, 809],
 [2948, 1048, 1529, 4031, 2392, 8],
 [174, 4754, 4881, 2657, 1133, 3145, 3539, 1759, 966],
 [1086, 2130, 547, 2793, 2839, 312, 4224, 413, 4494, 556],
 [447, 4328, 2794, 415, 475, 413, 4494, 556],
 [1568, 2091, 2451, 2539, 2033, 3644, 3164, 607, 2607, 3813],
 [1888, 1004, 170],
 [1164, 2319, 3387, 1937, 2392, 900, 63, 1604],
 [4174, 3567, 1678,

In [30]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [31]:
onehot_rep[1]

[4217, 170, 1678, 379, 4228, 779, 1604]

## **Embedding Representation (basically it is word 2 vec)**

In [32]:
#padding
sent_length=20
embedded_docs=pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4781 1896 3750]
 [   0    0    0 ... 4228  779 1604]
 [   0    0    0 ... 4962 3425 3356]
 ...
 [   0    0    0 ... 1942  502 4271]
 [   0    0    0 ... 1867  871 1879]
 [   0    0    0 ...  680 2392  415]]


In [33]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 4217,  170, 1678,  379, 4228,  779, 1604], dtype=int32)

In [34]:
from nltk.translate import metrics
## creating model
embedding_vector_features=40 # each word will get converted of vector 40 size(just play with any number-hyper parameter tuning)
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100)) # 100 neurons
model.add(Dense(1,activation='sigmoid')) # sigmoid-binary, Dense layer is 1
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary()) #embedding layer- 20 input, 40 output

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [35]:
len(embedded_docs),y.shape # first one is input and second one is output

(702, (702,))

In [36]:
import numpy as np
x_final=np.array(embedded_docs) #all the embedded form i see in the form of list(vectors) convert it into array
y_final=np.array(y)

In [37]:
x_final.shape,y_final.shape # (1297, 20)- 1297 input, 20- padding

((702, 20), (702,))

**Model Training**

In [38]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split( x_final, y_final, test_size=0.33, random_state=42)

In [39]:
### Finally model training
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
8/8 [==============================] - 3s 83ms/step - loss: 0.6868 - accuracy: 0.5745 - val_loss: 0.6674 - val_accuracy: 0.6164
Epoch 2/10
8/8 [==============================] - 0s 27ms/step - loss: 0.6579 - accuracy: 0.5638 - val_loss: 0.6287 - val_accuracy: 0.7241
Epoch 3/10
8/8 [==============================] - 0s 29ms/step - loss: 0.5891 - accuracy: 0.8511 - val_loss: 0.5745 - val_accuracy: 0.7716
Epoch 4/10
8/8 [==============================] - 0s 31ms/step - loss: 0.4479 - accuracy: 0.8745 - val_loss: 0.6941 - val_accuracy: 0.6767
Epoch 5/10
8/8 [==============================] - 0s 33ms/step - loss: 0.3830 - accuracy: 0.8213 - val_loss: 0.5602 - val_accuracy: 0.7672
Epoch 6/10
8/8 [==============================] - 0s 31ms/step - loss: 0.2843 - accuracy: 0.9596 - val_loss: 0.4896 - val_accuracy: 0.8362
Epoch 7/10
8/8 [==============================] - 0s 28ms/step - loss: 0.2245 - accuracy: 0.9830 - val_loss: 0.4554 - val_accuracy: 0.8405
Epoch 8/10
8/8 [===========

Dropout layer (improve the accuracy)

Performance Metrics and Accuracy

In [40]:
y_pred=model.predict(x_test)

8/8 [==============================] - 0s 4ms/step


In [41]:
#threshhold (limit)
y_pred=np.where(y_pred > 0.5,1,0) # when ever it is greater than 0 , i will take it as 1-fake and 0- real
#if i want to remove the thresh hold use (AUC ROC Curve)
#AUC ROC Curve - what is the probability you should select

In [42]:
from sklearn.metrics import confusion_matrix

In [43]:
confusion_matrix(y_test,y_pred)

array([[124,  19],
       [ 31,  58]])

In [44]:
# accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7844827586206896

In [45]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.80      0.87      0.83       143
           1       0.75      0.65      0.70        89

    accuracy                           0.78       232
   macro avg       0.78      0.76      0.77       232
weighted avg       0.78      0.78      0.78       232



In [46]:
# to increase accuracy use dropper,after applying run from model training (it will also reduce over fitting)
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])